# Running a Local Model

This notebook shows how to run a local model on your computer

We will be using llama CPP

References:
- https://github.com/abetlen/llama-cpp-python
- https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/llm/llama_2_llama_cpp.ipynb
- https://colab.research.google.com/drive/1UoPcoiA5EOBghxWKWduQhChliMHxla7U?usp=sharing#scrollTo=V1DwVkHCNDgT

## Step-1:  Installing LLama-CPP-Python

See here for detailed install guide: https://github.com/abetlen/llama-cpp-python

### 1.1 - Quick install for CPU 


```bash
# adjust this to match your environment name
conda activate atlas-1

pip install llama-cpp-python
```

### 1.2 - Quick install for GPU

For this you would need a Nvidia GPU with all the appropriate CUDA drivers installed.  How to install this varies from OS to OS.  ANd its beyond the scope of this guide.  So please make sure this is done, before you attempt to use GPU.

```bash
conda activate atlas-1

CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

# if you need to foce reinstall
CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --force-reinstall --no-cache-dir llama-cpp-python
```

## Step-2: How do I Know if the model is using GPU

To completely turn off GPU usage , set this variable before initializing the model

```python
os.environ["CUDA_VISIBLE_DEVICES"]=""
```

See below for checks we do for GPUs.

Keep in mind, even if code below says GPU is being used, unless llama-cpp-python is built with GPU support, the model won't use GPU.

So double check and verify!

### 2.1 - Check the logs when the model initialized

You can check the output when model is initialized as follows.  IF you see something like this, that means the model is using GPU!

```text
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 2070, compute capability 7.5, VMM: yes
```

### 2.2 - Use `nvidia-smi` tool

Use `nvidia-smi` tool (run this from terminal) to see if GPU memory is being used

Here is a sample output

![missing image](../images/nvidia-smi-1.png)

In [1]:
## Check if GPU is enabled
import os
import torch

## To disable GPU and experiment, uncomment the following line
## Normally, you would want to use GPU, if one is available.
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  True
device  0 NVIDIA GeForce RTX 2070


## Step-3: Choosing a Model

Running full sized models will require a lot of resources.  So we will be using **quantized** models.  These are 'slimmed down' models that are smaller, sacrificing some quality of output

See different options here : https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF

The smallest recommended model is 4 bit quantized: **Q4_K_M**  - it's a medium model that provides balanced quality.

So the filename is : **mistral-7b-instruct-v0.2.Q4_K_M.gguf**

Go ahead and download the model from here.

Here is a direct link to : [mistral-7b-instruct-v0.2.Q4_K_M.gguf](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/raw/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf)

![missing image](../images/mistral-model-selection-1.png)

In [2]:
import os

model_file_path = '../models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_M.gguf'
# model_file_path = '../models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q5_K_M.gguf'

## Step-4: Initialize the Model

Here we are going to init the model, with the following params:

- **model_path** : point to where the model file was downloaded
- **n_gpu_layers**: controls if model is off loaded into GPU
    - set to at least 1 to use GPU
    - change this value from 1, 10, 20, 30, 40
    - set to -1 to offload all layers to GPU
    - for Nvidia GEForce 2070 with 8 GB RAM 40 works well 
    - the `mistral-7b-instruct-v0.2.Q4_K_M.gguf` consumes about 5 GB of GPU RAM with full offload (-1)


In [3]:
from llama_cpp import Llama

llm = Llama(
      model_path=model_file_path,
      n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
    #   n_ctx=2048, # Uncomment to increase the context window
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ../models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 12

## Step-5: Let's test the model

In [10]:
%%time

import pprint

output = llm(
   "Q: What is the capital of France? A: ", # Prompt
   max_tokens=32,  # Generate up to this many tokens, set to None to generate up to the end of the context window
   stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
   echo=False  # Echo the prompt back in the output
)
## Note  how the answer is truncated, after reaching 'max_tokens' ?
## Set the max_tokens to None and see what you get
print ('-----', flush=True)
print (output['choices'][0]['text'])
print ('-----', flush=True)
pprint.pprint (output, indent=4)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     119.85 ms
llama_print_timings:      sample time =      10.14 ms /    32 runs   (    0.32 ms per token,  3155.51 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     587.17 ms /    32 runs   (   18.35 ms per token,    54.50 tokens per second)
llama_print_timings:       total time =     653.40 ms /    33 tokens


-----
 Paris, officially known as the City of Paris, is the capital city of France. It is a major European political and cultural centre, and one of the world
-----
{   'choices': [   {   'finish_reason': 'length',
                       'index': 0,
                       'logprobs': None,
                       'text': ' Paris, officially known as the City of Paris, '
                               'is the capital city of France. It is a major '
                               'European political and cultural centre, and '
                               'one of the world'}],
    'created': 1711177782,
    'id': 'cmpl-fab4bce3-927b-479e-9877-7d73dc262f3b',
    'model': '../models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    'object': 'text_completion',
    'usage': {'completion_tokens': 32, 'prompt_tokens': 13, 'total_tokens': 45}}
CPU times: user 641 ms, sys: 15.3 ms, total: 656 ms
Wall time: 657 ms


In [5]:
%%time

import pprint

output = llm(
   # "Q: Name the planets in the Solar system.  A: ",
   "Name the planets in the Solar system.",
   max_tokens=None,
   #stop=["Q:"],
   echo=False
)

print ('-----', flush=True)
print (output['choices'][0]['text'])
print ('-----', flush=True)
pprint.pprint (output, indent=4)


Llama.generate: prefix-match hit

llama_print_timings:        load time =     119.85 ms
llama_print_timings:      sample time =      53.44 ms /   154 runs   (    0.35 ms per token,  2881.74 tokens per second)
llama_print_timings: prompt eval time =      58.92 ms /     8 tokens (    7.37 ms per token,   135.78 tokens per second)
llama_print_timings:        eval time =    2617.43 ms /   153 runs   (   17.11 ms per token,    58.45 tokens per second)
llama_print_timings:       total time =    3012.81 ms /   161 tokens


-----
 I'll give you a hint: There are eight of them, and they all revolve around the Sun.

Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune. Each planet has unique characteristics that set it apart from the others. Mercury is the smallest and closest to the sun, while Neptune is the farthest and largest. Earth is the only known planet to harbor life. Jupiter is the largest planet, and Saturn is known for its prominent ring system. Uranus is tilted on its axis, while Mars is called the Red Planet due to its reddish appearance. All eight planets provide fascinating insights into the wonders of our universe.
-----
{   'choices': [   {   'finish_reason': 'stop',
                       'index': 0,
                       'logprobs': None,
                       'text': " I'll give you a hint: There are eight of "
                               'them, and they all revolve around the Sun.\n'
                               '\n'
                               'Mercury, Venus, Earth

In [6]:
%%time

import pprint

output = llm(
   #"Q: Write a Haiku about cats.  A: ",
   "Write a Haiku about cats.",
   max_tokens=None,
   #stop=["Q:", "\n"],
   echo=False
)

print ('-----', flush=True)
print (output['choices'][0]['text'])
print ('-----', flush=True)
pprint.pprint (output, indent=4)


Llama.generate: prefix-match hit

llama_print_timings:        load time =     119.85 ms
llama_print_timings:      sample time =      10.05 ms /    30 runs   (    0.34 ms per token,  2985.07 tokens per second)
llama_print_timings: prompt eval time =      48.55 ms /     7 tokens (    6.94 ms per token,   144.20 tokens per second)
llama_print_timings:        eval time =     499.06 ms /    29 runs   (   17.21 ms per token,    58.11 tokens per second)
llama_print_timings:       total time =     612.97 ms /    36 tokens


-----


Soft, stealthy paws tread,
Silent hunters in night,
Curious, playful feline.
-----
{   'choices': [   {   'finish_reason': 'stop',
                       'index': 0,
                       'logprobs': None,
                       'text': '\n'
                               '\n'
                               'Soft, stealthy paws tread,\n'
                               'Silent hunters in night,\n'
                               'Curious, playful feline.'}],
    'created': 1711177709,
    'id': 'cmpl-499f7986-21d3-4a63-ae32-30890950a00c',
    'model': '../models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    'object': 'text_completion',
    'usage': {'completion_tokens': 29, 'prompt_tokens': 8, 'total_tokens': 37}}
CPU times: user 616 ms, sys: 2.81 ms, total: 619 ms
Wall time: 617 ms
